In [1]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import mysql.connector
import requests

from pydantic import BaseModel
import pickle

import pandas as pd
import math
import json
import re

In [18]:
# python backend.py

In [3]:
#PARA DETENER EL PROCESO:

#netstat -ano | findstr :8080

#me sale un número después de "LISTENING" que es el PID

#taskkill /F /PID número

Obtengo una dirección IP local: (cogí la que pone IPv4)

In [1]:
!ipconfig


Configuraci¢n IP de Windows


Adaptador de Ethernet Ethernet:

   Estado de los medios. . . . . . . . . . . : medios desconectados
   Sufijo DNS espec¡fico para la conexi¢n. . : 

Adaptador de LAN inal mbrica Conexi¢n de  rea local* 9:

   Estado de los medios. . . . . . . . . . . : medios desconectados
   Sufijo DNS espec¡fico para la conexi¢n. . : 

Adaptador de LAN inal mbrica Conexi¢n de  rea local* 10:

   Estado de los medios. . . . . . . . . . . : medios desconectados
   Sufijo DNS espec¡fico para la conexi¢n. . : 

Adaptador de LAN inal mbrica Wi-Fi:

   Sufijo DNS espec¡fico para la conexi¢n. . : alumnos.uem.es
   V¡nculo: direcci¢n IPv6 local. . . : fe80::4662:39d9:9c79:f8f8%6
   Direcci¢n IPv4. . . . . . . . . . . . . . : 10.100.16.136
   M scara de subred . . . . . . . . . . . . : 255.255.224.0
   Puerta de enlace predeterminada . . . . . : 10.100.0.1

Adaptador de Ethernet Conexi¢n de red Bluetooth:

   Estado de los medios. . . . . . . . . . . : medios desconectados
   S

Importacion de los modelos:

In [8]:
with open('kmode.pkl','rb') as km:
    kmode= pickle.load(km)
    
with open('filtrado.pkl','rb') as f:
    filtrado= pickle.load(f)

Definición de funciones para cada modelo:

In [ ]:
## METER LAS FUNCIONES QUE USA CADA MODELO AQUÍ

Definición de funciones FastApi:

In [21]:
# Creo una instancia de FastApi
app = FastAPI()

In [ ]:
@app.get("/")
async def root():
    return {"message": "Hola mundo"}

In [ ]:
# Esta solamente imprime los valores que me da el usuario

@app.get("/recogerdatos1/")
def recoger_datos(especie: str, raza: str, sexo: str):
    # Retornar una respuesta con los datos recibidos
    
    print("Especie:", especie)
    print("Raza:", raza)
    print("Sexo:", sexo)
    
    return {"message": "Datos recibidos correctamente", "especie": especie, "raza": raza, "sexo": sexo}

In [ ]:
# Esta función me pasa los datos del df de centros

@app.get("/enviarcentros/")
def get_centros():
    # Cargar dataframe
    df= pd.read_csv("./poblacion_centros_v3.csv")

    # Convertir el dataframe a formato JSON
    json_data = df.to_json(orient="records")

    # Devolver los datos en una respuesta JSON
    return JSONResponse(content=json_data)

In [3]:
df= pd.read_csv("./poblacion_centros_v3.csv")

In [9]:
df_cleaned = df.replace([math.inf, -math.inf], math.nan)  # Reemplazar infinitos con NaN

In [12]:
df_cleaned.isnull().any().sum()

1

In [13]:
df_cleaned = df_cleaned.dropna()

In [14]:
df_cleaned.isnull().any().sum()

0

In [16]:
json_data = df.to_json(orient="records")

In [20]:
type(JSONResponse(content=json.loads(json.dumps(json_data))))

starlette.responses.JSONResponse

In [17]:
json_data

'[{"Nombre":"HELPING ANIMALS ALMANZORA","Direccion":"C\\/ HUELVA N\\u00ba 5 , Arboleas (ALMERIA) , Andaluc\\u00eda, Espa\\u00f1a","CCAA":"Andaluc\\u00eda","Latitud":37.34997,"Longitud":-2.0737284,"Telefono":"+34877 393 789","URL":"http:\\/\\/www.helpinganimalsalmanzora.com","Correo":"helpinga@reguera.com","Facebook":"register\\/helpinganimalsalmanzora","Categoria":"mixta","Tama\\u00f1o":"mediana","Capacidad":253,"Ocupacion":252,"id_centro":1},{"Nombre":"PROTECTORA DE ANIMALES ROCIANA ACOGE","Direccion":"CL VENDIMIADORES 33 , Rociana del Condado (HUELVA) , Andaluc\\u00eda, Espa\\u00f1a","CCAA":"Andaluc\\u00eda","Latitud":37.3121342,"Longitud":-6.5954551,"Telefono":"+34823 95 92 00","URL":"http:\\/\\/www.protectoradeanimalesrocianaacoge.com","Correo":"prote@castaneda.net","Facebook":"author\\/protectoradeanimalesrocianaacoge","Categoria":"mixta","Tama\\u00f1o":"mediana","Capacidad":151,"Ocupacion":104,"id_centro":2},{"Nombre":"ABRAZO ANIMAL JAMILENA","Direccion":"C\\/ MORERIA, 27 , Marto

In [18]:
type(json_data)

str

In [11]:
# 1. Pasar fechas input al formato de los datos

def combinar_edad(anios, meses):
    anios_str = str(anios) + " año" + ("s" if anios != '1' else "")
    meses_str = str(meses) + " mes" + ("es" if meses != '1' else "")
    
    if anios != 0 and meses != 0:
        return anios_str + " y " + meses_str
    elif anios != 0:
        return anios_str
    elif meses != 0:
        return meses_str
    else:
        return "0 meses"  # Si ambos son cero, retorna "0 meses"

# 2. Parseo fechas

def parse_age(age_str):
    match = re.search(r'(\d+) años? (y (\d+) meses?)?', age_str)
    if match:
        years_str = match.group(1)
        years = int(years_str) if years_str else 0
        months_str = match.group(3)
        months = int(months_str) if months_str else 0
        total_months = years * 12 + months
        return total_months / 12
    else:
        return None



# FUNCION MODELO 1 (FASTAPI)

especie= 'Canina'
raza= 'mestizo'
sexo= 'macho'
tamano= 'grande'
microchip= 'false'
years_tiempo= '1'
months_tiempo= '3'
years_edad= '4'
months_edad= '2'

#@app.get("/recogerdatos1/")
#def recoger_datos(especie: str, raza: str, sexo: str, tamano: str, microchip: str, years_tiempo: str, months_tiempo: str, years_edad: str, months_edad: str):
def recoger_datos(especie, raza, sexo, tamano, microchip, years_tiempo, months_tiempo, years_edad, months_edad):
    
    # Carga de los dataframes que se van a utilizar
    dataframe= pd.read_csv("./poblacion_adoptados_v3.csv")
    centros= pd.read_csv("./poblacion_centros_v3.csv")  
    
    # Conversion de fechas input a formato de los datos
    tiempo_combinado= combinar_edad(years_tiempo, months_tiempo)
    años_combinado= combinar_edad(years_edad, months_edad)
    
    # Diccionario datos input
    user_input = {'id_animal':str(len(dataframe) + 1), 'id_centro':'', 'Raza':raza,
                  'Especie':especie,'Sexo': sexo,'Tamaño':tamano,
                  'Microchip':bool(microchip), 'Tiempo':tiempo_combinado,
                  'Edad':años_combinado}
    
    # Añadir input a dataframe en la ultima posicion
    dataframe_completo = dataframe.append(user_input, ignore_index=True)

    # Parseo de fechas de todo el dataframe
    dataframe_completo['Tiempo_parseado'] = dataframe_completo['Tiempo'].apply(parse_age)
    dataframe_completo['Edad_parseado'] = dataframe_completo['Edad'].apply(parse_age)
    
    # Seleccion de las columnas de entrada al modelo
    dataframe_modelo= dataframe_completo[['Raza', 'Especie', 'Sexo', 'Tamaño',
                                          'Microchip', 'Tiempo_parseado', 'Edad_parseado']]

    # Algoritmo
    clusters= kmode.fit_predict(dataframe_modelo)

    # Añadir la columna de los clusters al dataframe con todas las variables
    dataframe_completo.insert(0, "Cluster", clusters , True)
    
    # ¿A qué cluster pertenece la última fila del dataframe?
    numCluster= dataframe_completo.iloc[-1]['Cluster']

    # Borrar la última fila del dataframe (input) para trabajar con el resto de datos
    dataframe_completo= dataframe_completo[:-1]
    
    # De los datos iniciales, ¿Cuales pertenecen al mismo cluster que el input?
    mascara = dataframe_completo.loc[:, 'Cluster'] == numCluster
    mismoCluster= dataframe_completo[mascara]

    # Del mismo cluster, ¿Qué número de registros hay para cada centro?
    frecuencia_serie= mismoCluster.groupby('id_centro').size()
    sorted_serie = frecuencia_serie.sort_values(ascending= False)  
    sorted_df = pd.DataFrame(sorted_serie, columns=['frecuencia'])
    
    #¿Con qué centro se corresponde cada id? <- ORDENADOS
    df_ok= pd.merge(sorted_df,centros, on= 'id_centro')
    
    # Por si acaso
    df_ok = df_ok.dropna()
    
    return df_ok
    
    # Convertir el dataframe a formato JSON
    #json_data = df_ok.to_dict(orient="records")
    
    # Devolver los datos en una respuesta JSON
    #return JSONResponse(content=json_data)

In [13]:
respuesta= recoger_datos(especie, raza, sexo, tamano, microchip, years_tiempo, months_tiempo, years_edad, months_edad)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_2748\955108817.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_completo = dataframe.append(user_input, ignore_index=True)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 51738, cost: 1014827.0


In [17]:
json_data= respuesta.to_dict(orient= "records")

In [18]:
json_data

[{'id_centro': 743,
  'frecuencia': 1095,
  'Nombre': 'RESCATE ANIMAL MARINA ALTA (R.A.M.A)',
  'Direccion': 'PLAÇA DE BAIX, 5 Xàbia/Jávea ALICANTE Valencia',
  'CCAA': 'Valencia',
  'Latitud': 38.78895795,
  'Longitud': 0.163723961557172,
  'Telefono': '+34887 95 64 28',
  'URL': 'http://www.rescateanimalmarinaaltar.a.m.a.com',
  'Correo': 'resca@porras.es',
  'Facebook': 'login/rescateanimalmarinaaltar.a.m.a',
  'Categoria': 'canina',
  'Tamaño': 'grande',
  'Capacidad': 499,
  'Ocupacion': 892},
 {'id_centro': 650,
  'frecuencia': 999,
  'Nombre': 'Associació Progat Terrassa',
  'Direccion': 'Carrer Goleta, 11 8221 Terrassa Barcelona Vallès Occidental Cataluña',
  'CCAA': 'Cataluña',
  'Latitud': 41.5629792,
  'Longitud': 2.0081779,
  'Telefono': '+34 807933070',
  'URL': 'http://www.associaciprogatterrassa.com',
  'Correo': 'associacip@arnaiz.com',
  'Facebook': 'faq/associaciprogatterrassa',
  'Categoria': 'felina',
  'Tamaño': 'grande',
  'Capacidad': 478,
  'Ocupacion': 867},
 {

In [15]:
respuesta.isnull().any()

id_centro     False
frecuencia    False
Nombre        False
Direccion     False
CCAA          False
Latitud       False
Longitud      False
Telefono      False
URL           False
Correo        False
Facebook      False
Categoria     False
Tamaño        False
Capacidad     False
Ocupacion     False
dtype: bool

In [2]:
df= pd.read_csv("./poblacion_adopcion_v3.csv")

In [3]:
df['Raza'].unique()

array(['Pastor Belga', 'Mestizo', 'Podenco', 'Gato', 'Cruce Podenco',
       'Cruce Mastín', 'Desconocida', 'Cruce Sharpei', 'Cruce Pointer',
       'Galgo', 'Yorkshire Terrier', 'Cruce Bodeguero',
       'Cruce Pastor Belga', 'Bodeguero', 'Cruce Bull Terrier',
       'Cruce Fox Terrier', 'Bretón Español', 'Mastín',
       'Cruce Pastor Alemán', 'Cruce Labrador', 'Cruce PPP',
       'Pastor Alemán', 'Bichón Maltés', 'Cruce Basenji', 'Cruce Husky',
       'Cruce Pastor del Cáucaso', 'Perro de agua'], dtype=object)

In [4]:
df.columns

Index(['Nombre', 'Sexo', 'Raza', 'Edad', 'Tamaño', 'Especie', 'id_adopcion',
       'cat_abandonado', 'cat_cariño', 'cat_cercania', 'cat_acogida',
       'cat_actividad', 'cat_dependencia', 'cat_adaptacion', 'cat_docil',
       'cat_agradecido', 'cat_agresivo', 'cat_tenso', 'cat_alegre',
       'cat_inteligencia', 'cat_miedo', 'cat_convivencia', 'cat_correa',
       'cat_carretera', 'cat_aventura', 'cat_niños', 'cat_belleza', 'cat_ppp',
       'cat_bruto', 'cat_triste', 'cat_tranquilidad', 'cat_desubicado',
       'cat_recuperacion', 'cat_cronico', 'cat_historial', 'cat_fortaleza',
       'cat_independencia', 'cat_maltrato', 'cat_gato', 'cat_perro',
       'cat_pelo', 'cat_macho', 'cat_hembra', 'cat_ladra', 'cat_ruido',
       'id_centro'],
      dtype='object')

In [5]:
df2= pd.read_csv("./poblacion_centros_v3.csv")

In [6]:
df2.columns

Index(['Nombre', 'Direccion', 'CCAA', 'Latitud', 'Longitud', 'Telefono', 'URL',
       'Correo', 'Facebook', 'Categoria', 'Tamaño', 'Capacidad', 'Ocupacion',
       'id_centro'],
      dtype='object')